#Colab 실습

좌측에 구글 드라이브 표시 클릭 후 구글 드라이브 마운트를 해줌. 이후에 PPT 링크에서 다운받은 두 파일을 구글 드라이브 내의 적절한 위치에 업로드 필요. 이후 코드에서 마운트된 구글 드라이브 내의 파일을 사용할 예정.

In [ ]:
#FairMOT 다운로드
!git clone https://github.com/sihwa-park/FairMOT.git

Cloning into 'FairMOT'...
remote: Enumerating objects: 569, done.
remote: Total 569 (delta 0), reused 0 (delta 0), pack-reused 569
Receiving objects: 100% (569/569), 134.86 MiB | 41.77 MiB/s, done.
Resolving deltas: 100% (289/289), done.


In [ ]:
#Human_Falling_Detect_Tracks 다운로드
!git clone https://github.com/sihwa-park/Human-Falling-Detect-Tracks.git Human_Falling_Detect_Tracks

Cloning into 'Human_Falling_Detect_Tracks'...
remote: Enumerating objects: 178, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 178 (delta 3), reused 0 (delta 0), pack-reused 167
Receiving objects: 100% (178/178), 168.35 MiB | 39.92 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [ ]:
#FairMOT에서 필요한 라이브러리들 설치
!pip install -r FairMOT/requirements.txt

     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 153kB 20.1MB/s 
     |████████████████████████████████| 1.5MB 45.1MB/s 
     |████████████████████████████████| 122kB 56.3MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 81kB 12.3MB/s 
     |████████████████████████████████| 102kB 12.0MB/s 
     |████████████████████████████████| 71kB 11.2MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
  Created wheel for cython-bbox: filename=cython_bbox-0.1.3-cp37-cp37m-linux_x86_64.whl size=57049 sha256=d8d4e38ba273c5fe2a90c0c4b427eb5a04d5d1e2136165d75d71ec3480d3120a
  Stored in directory: /root/.cache/pip/wheels/2b/31/b5/9246d5988e79ef89dc28b894835d2f305e23c1e5f4f80278ee
  Created wheel for progress: filename=progress-1.5-cp37-none-any.whl size=8074 sha256=1895b9184e71d743559267d071c7c9ee4c9f09d8abc7172aef8c46802936819a
  Stored in directory: /root/.cache/pip/wheels/6c/c8/80/32a294e3041f006c66

In [ ]:
#pytorch 버전 고정
!python3 -m pip install torch==1.5.0 torchvision==0.6.0

     |████████████████████████████████| 752.0MB 20kB/s 
     |████████████████████████████████| 6.6MB 30.9MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.5.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101
  Found existing installation: torchvision 0.9.1+cu101
    Uninstalling torchvision-0.9.1+cu101:
      Successfully uninstalled torchvision-0.9.1+cu101


In [ ]:
#FairMOT를 위한 다른 라이브러리 설치
!git clone https://github.com/lbin/DCNv2

Cloning into 'DCNv2'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 263 (delta 9), reused 12 (delta 6), pack-reused 240
Receiving objects: 100% (263/263), 1.44 MiB | 28.88 MiB/s, done.
Resolving deltas: 100% (151/151), done.


In [ ]:
!python DCNv2/setup.py build develop

running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/__init__.py -> build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/dcn_v2.py -> build/lib.linux-x86_64-3.7/DCNv2
copying DCNv2/setup.py -> build/lib.linux-x86_64-3.7/DCNv2
running build_ext
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:304: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
building '_ext' extension
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/content
creating build/temp.linux-x86_64-3.7/content/DCNv2
creating build/temp.linux-x86_64-3.7/content/DCNv2/src
creating build/temp.linux-x86_64-3.7/content/DCNv2/src/cpu
creating build/temp.linux-x86_64-3.7/content/DCNv2/src/cuda
g++ -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -

In [ ]:
%cd DCNv2/

/content/DCNv2


In [ ]:
import sys

sys.path.append('/content/FairMOT/src/lib')
sys.path.append('/content/Human_Falling_Detect_Tracks')
sys.path.insert(0, '/content')

CCTV 이상상황 탐지 모델에 사용하는 모듈들은 import 해줍니다.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import time
import os
import os.path as osp
import cv2
import numpy as np
import torch
import datetime
import copy

from tracker.multitracker import JDETracker
from tracking_utils import visualization as vis
from tracking_utils.log import logger
from tracking_utils.timer import Timer
from tracking_utils.evaluation import Evaluator
import datasets.dataset.jde as datasets

from tracking_utils.utils import mkdir_if_missing
from opts import opts
from shapely.geometry.polygon import Polygon
import threading

from Human_Falling_Detect_Tracks.Detection.Utils import ResizePadding
from Human_Falling_Detect_Tracks.DetectorLoader import TinyYOLOv3_onecls
from Human_Falling_Detect_Tracks.PoseEstimateLoader import SPPE_FastPose
from Human_Falling_Detect_Tracks.Track.Tracker import Detection, Tracker
from Human_Falling_Detect_Tracks.ActionsEstLoader import TSSTG

각종 함수들을 선언해줍니다.

In [ ]:
def letterbox(img, height=608, width=1088, color=(127.5, 127.5, 127.5)):  # resize a rectangular image to a padded rectangular
    shape = img.shape[:2]  # shape = [height, width]
    ratio = min(float(height) / shape[0], float(width) / shape[1])
    new_shape = (round(shape[1] * ratio), round(shape[0] * ratio))  # new_shape = [width, height]
    dw = (width - new_shape[0]) / 2  # width padding
    dh = (height - new_shape[1]) / 2  # height padding
    top, bottom = round(dh - 0.1), round(dh + 0.1)
    left, right = round(dw - 0.1), round(dw + 0.1)
    img = cv2.resize(img, new_shape, interpolation=cv2.INTER_AREA)  # resized, no border
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # padded rectangular
    return img, ratio, dw, dh

def in_area(areas, description, tlwh):
    x1, y1, w, h = tlwh #x1, y1 is left top
    x2, y2 = x1 + w, y1 + h # x2, y2 is right bottom
    area = areas[description]
    input_area = Polygon([(x1, y1), (x1, y2), (x2, y2), (x2, y1)])
    if area.area > 0:
        if area.contains(input_area):
            return True
        if input_area.intersects(area):
            #print((input_area.intersection(area).area/input_area.area)*100)
            if (input_area.intersection(area).area/input_area.area)*100 > 90:
                return True
    return False

def is_in_walking_people(walking_people, x1, y1, x2, y2, thre = 20):
    for pid in list(walking_people.keys()):
        tlwh = walking_people[pid].tlwhs[-1]
        walking_x1, walking_y1, walking_w, walking_h = tlwh
        walking_x2, walking_y2 = walking_x1 + walking_w, walking_y1 + walking_h
        walking_area = Polygon([(walking_x1, walking_y1), (walking_x1, walking_y2), (walking_x2, walking_y2), (walking_x2, walking_y1)])
        area = Polygon([(x1, y1), (x1, y2), (x2, y2), (x2, y1)])
        if area.intersects(walking_area):
            if (area.intersection(walking_area).area/area.area)*100 > thre:
                return (pid, (area.intersection(walking_area).area/area.area)*100)
    return (False, 0)

def is_in_falldown_people(falldown_people, x1, y1, x2, y2, thre = 50):
    for pid in list(falldown_people.keys()):
        tlwh = falldown_people[pid].tlwhs[-1]
        walking_x1, walking_y1, walking_w, walking_h = tlwh
        walking_x2, walking_y2 = walking_x1 + walking_w, walking_y1 + walking_h
        walking_area = Polygon([(walking_x1, walking_y1), (walking_x1, walking_y2), (walking_x2, walking_y2), (walking_x2, walking_y1)])
        area = Polygon([(x1, y1), (x1, y2), (x2, y2), (x2, y1)])
        if area.area > 0 and area.intersects(walking_area):
            #print('is_in_falldown_people', pid, (area.intersection(walking_area).area/area.area)*100)
            if (area.intersection(walking_area).area/area.area)*100 > thre:
                return (pid, (area.intersection(walking_area).area/area.area)*100)
    return (False, 0)

def is_near_border(tlwh):
    x1, y1, w, h = tlwh
    x2, y2 = x1 + w, y1 + h
    area = Polygon([(x1, y1), (x1, y2), (x2, y2), (x2, y1)])
    center = Polygon([(100,0), (100,636), (1180,636), (1180,100)])
    if area.intersects(center):
        if (area.intersection(center).area/area.area)*100 > 90:
            return False
    return True

def plot_tracking(image, tlwhs, obj_ids, scores=None, frame_id=0, fps=0., ids2=None):
    im = np.ascontiguousarray(np.copy(image))
    im_h, im_w = im.shape[:2]

    top_view = np.zeros([im_w, im_w, 3], dtype=np.uint8) + 255

    text_scale = max(1, image.shape[1] / 1600.)
    text_thickness = 2
    line_thickness = max(1, int(image.shape[1] / 500.))

    radius = max(5, int(im_w/140.))
#     cv2.putText(im, 'frame: %d fps: %.2f num: %d' % (frame_id, fps, len(tlwhs)),
#                 (0, int(15 * text_scale)), cv2.FONT_HERSHEY_PLAIN, text_scale, (0, 0, 255), thickness=2)

    for i, tlwh in enumerate(tlwhs):
        x1, y1, w, h = tlwh
        intbox = tuple(map(int, (x1, y1, x1 + w, y1 + h)))
        obj_id = int(obj_ids[i])
        id_text = '{}'.format(int(obj_id))
        if ids2 is not None:
            id_text = id_text + ', {}'.format(int(ids2[i]))
        _line_thickness = 1 if obj_id <= 0 else line_thickness
        color = vis.get_color(abs(obj_id))
        cv2.rectangle(im, intbox[0:2], intbox[2:4], color=color, thickness=line_thickness)
        cv2.putText(im, id_text, (intbox[0], intbox[1] + 30), cv2.FONT_HERSHEY_PLAIN, text_scale, (0, 0, 255),
                    thickness=text_thickness)
    return im

def kpt2bbox(kpt, ex=20):
    """Get bbox that hold on all of the keypoints (x,y)
    kpt: array of shape `(N, 2)`,
    ex: (int) expand bounding box,
    """
    return np.array((kpt[:, 0].min() - ex, kpt[:, 1].min() - ex,
                     kpt[:, 0].max() + ex, kpt[:, 1].max() + ex))

In [ ]:
import imageio

def file_upload(category, alarm, dt_str):
    # start_time = time.time()
    # total_images = 0
    files = {}
    if category == 'intrusion' or category == 'loitering':
        original_images_list = []
        original_filenames_list = []
        tagged_images_list = []
        tagged_filenames_list = []
        original_filename = category + "_original_" + dt_str + ".gif"
        tagged_filename = category + "_tagged_" + dt_str + ".gif"
        for case in alarm.cases:
            for filename in case.original_image_list:
                original_filenames_list.append(filename)
            for filename in case.tagged_image_list:
                tagged_filenames_list.append(filename)
        original_filenames_list = list(set(original_filenames_list))
        tagged_filenames_list = list(set(tagged_filenames_list))
        # total_images = len(original_filenames_list)
        original_filenames_list.sort()
        tagged_filenames_list.sort()
        for filename in original_filenames_list:
            original_images_list.append(imageio.imread(filename))
        for filename in tagged_filenames_list:
            tagged_images_list.append(imageio.imread(filename))
        imageio.mimsave(os.path.join('../', original_filename), original_images_list, fps=30)
        imageio.mimsave(os.path.join('../', tagged_filename), tagged_images_list, fps=30)
    elif category == 'falldown':
        original_filename = category + "_original_" + dt_str + ".gif"
        original_images_list = []
        # total_images = len(alarm.image_list)
        for image in alarm.image_list:
            original_images_list.append(imageio.imread(image))
        imageio.mimsave(os.path.join('../', original_filename), original_images_list, fps=30)

In [ ]:
class Alarm(object):
    def __init__(self, description, start_time, start_frame):
        self.description = description
        self.start_time = start_time
        self.start_frame = start_frame


class FalldownAlarm(Alarm):
    def __init__(self, description, start_frame, track_id, tlwhs):
        self.description = description
        self.start_frame = start_frame
        self.track_id = track_id
        self.tlwhs = [tlwhs]
        self.cases = []
        self.image_list = []

class LoiteringAlarm(Alarm):
    def __init__(self, description, start_time, start_frame):
        self.description = description
        self.cases = []
        self.start_time = start_time
        self.end_time = start_time
        self.start_frame = start_frame
        self.end_frame = start_frame

class IntrusionAlarm(Alarm):
     def __init__(self, description, start_time, start_frame):
        self.description = description
        self.cases = []
        self.start_time = start_time
        self.end_time = start_time
        self.start_frame = start_frame
        self.end_frame = start_frame


class Walking(object):
    def __init__(self, oid, start_frame, tlwhs, original_image_list, tagged_image_list):
        self.id = oid
        self.start_frame = start_frame
        self.end_frame = start_frame
        self.tlwhs = [tlwhs]
        self.original_image_list = original_image_list
        self.tagged_image_list = tagged_image_list


class Loitering(object):
    def __init__(self, oid, start_time, start_frame, tlwhs, original_image_path, tagged_image_path):
        self.id = oid
        self.start_time = start_time
        self.end_time = None
        self.start_frame = start_frame
        self.end_frame = start_frame
        self.is_alarm = False
        self.tlwhs = [tlwhs]
        self.original_image_list = [original_image_path]
        self.tagged_image_list = [tagged_image_path]

class Intrusion(object):
     def __init__(self, oid, start_time, start_frame, tlwhs, original_image_path, tagged_image_path):
        self.id = oid
        self.start_time = start_time
        self.end_time = None
        self.start_frame = start_frame
        self.end_frame = start_frame
        self.is_alarm = False
        self.tlwhs = [tlwhs]
        self.original_image_list = [original_image_path]
        self.tagged_image_list = [tagged_image_path]

pretrained fairmot model 다운로드


In [ ]:
!gdown --id 1bS_ZP1zKIdwKmlDftsNPSS1cHflyMUB2

Downloading...
From: https://drive.google.com/uc?id=1bS_ZP1zKIdwKmlDftsNPSS1cHflyMUB2
To: /content/DCNv2/fairmot_dla34.pth
259MB [00:04, 52.6MB/s]


In [ ]:
opt = opts().init(args=['mot', '--input_w', '1280', '--input_h', '736', '--load_model',
                                '/content/DCNv2/fairmot_dla34.pth', '--conf_thres', '0.4', '--track_buffer', '150'])


Fix size testing.
training chunk_sizes: [6, 6]
The output will be saved to  /content/FairMOT/src/lib/../../exp/mot/default
heads {'hm': 1, 'wh': 4, 'id': 128, 'reg': 2}


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

human falling detect pretrained model 다운로드


In [ ]:
!gdown --id 141dL6ChcEaDeju_xFGObEOVIY9kA_rZz

Downloading...
From: https://drive.google.com/uc?id=141dL6ChcEaDeju_xFGObEOVIY9kA_rZz
To: /content/DCNv2/fast_res50_256x192.pth
163MB [00:00, 200MB/s]


In [ ]:
opt.device = torch.device('cuda')
frame_rate = 10
tracker = JDETracker(opt, frame_rate=frame_rate)
# 쓰러짐 관련 사람 detection 모델
inp_dets = 384
detect_model = TinyYOLOv3_onecls(inp_dets, device=opt.device, config_file='/content/Human_Falling_Detect_Tracks/Models/yolo-tiny-onecls/yolov3-tiny-onecls.cfg', weight_file='/content/Human_Falling_Detect_Tracks/Models/yolo-tiny-onecls/best-model.pth')
# POSE MODEL.
inp_pose = '224x160'.split('x')
inp_pose = (int(inp_pose[0]), int(inp_pose[1]))
pose_model = SPPE_FastPose('resnet50', inp_pose[0], inp_pose[1], device=opt.device, weight_file='/content/DCNv2/fast_res50_256x192.pth')

# 쓰러짐 관련 Tracker.
max_age = 30
action_tracker = Tracker(max_age=max_age, n_init=3)

# Actions Estimate.
action_model = TSSTG(weight_file='/content/Human_Falling_Detect_Tracks/Models/TSSTG/tsstg-model.pth')

resize_fn = ResizePadding(inp_dets, inp_dets)
# 탐지 영역 설정
detectAreaFull = [(0, 0), (0, 736), (1280, 736), (1280, 0)]
areas = {}
# 탐지 영역을 전체화면으로 설정
areas['intrusion'] = Polygon(detectAreaFull)
areas['loitering'] = Polygon(detectAreaFull)
areas['general'] = Polygon(detectAreaFull)
# _people 이 붙은 변수 명은 현재 해당 레이블 영역에 있는 사람 정보입니다.
people = {}
walking_people = {}
falldown_people = {}
loitering_people = {}
intrusion_people = {}
# 이상상황이 탐지되었을 때 alarms 변수에 추가됩니다.
alarms = []
loitering_alarms = []
intrusion_alarms = []
first_capture = 0
timer = Timer()
frame_id = 0
mkdir_if_missing('../frames')
mkdir_if_missing('../falldown')
mkdir_if_missing('../frames/original')
mkdir_if_missing('../frames/tagged')

Creating model...
loaded /content/DCNv2/fairmot_dla34.pth, epoch 30
Loading pose model from /content/DCNv2/fast_res50_256x192.pth


In [ ]:
rtsp_address = "rtsp://wowzaec2demo.streamlock.net/vod/mp4:BigBuckBunny_115k.mov"
cap = cv2.VideoCapture(rtsp_address)
cap.isOpened()

True

In [ ]:
over_max_tries = 0
while True:
  ret, frame = cap.read()
  if not cap.isOpened():
    tries = 0
    max_tries = 100
    while ret == False:
      print("Can't receive frame. Retrying ...")
      cap.release()
      cap = cv2.VideoCapture(rtsp_address)
      ret, frame = cap.read()
      tries += 1
      if tries > max_tries:
        print("Over max tries")
        over_max_tries = 1
        break
  if over_max_tries == 1:
    break
  if ret:
    if frame_id > 1000:
      break
    timer.tic()
    frame = cv2.resize(frame, (1280, 720), fx=0, fy=0, interpolation=cv2.INTER_CUBIC)
    #if time_elapsed > 1./frame_rate:
    # prev = time.time()
    # 현재 시간을 파일명으로 이미지 파일을 저장합니다.
    dt_obj = datetime.datetime.now()
    dt_str = dt_obj.strftime("%y%m%d%H%M%S%f")
    if frame_id % 100 == 0:
      print('Processing frame {} ({:.2f} fps)'.format(frame_id, 1. / max(1e-5, timer.average_time)))
    
    # Padded resize
    img, _, _, _ = letterbox(frame, height=608, width=1088)
    # Normalize RGB
    img = img[:, :, ::-1].transpose(2, 0, 1)
    img = np.ascontiguousarray(img, dtype=np.float32)
    img /= 255.0

    blob = torch.from_numpy(img).cuda().unsqueeze(0)
    # blob = torch.from_numpy(img).unsqueeze(0)
    online_targets = tracker.update(blob, frame)
    online_tlwhs = []
    online_ids = []
    # 최소 영역 이상인 타겟들만 한 번 걸러줍니다.
    for idx, t in enumerate(online_targets):
        tlwh = t.tlwh
        tid = t.track_id
        if tlwh[2] * tlwh[3] > opt.min_box_area:
            online_tlwhs.append(tlwh)
            online_ids.append(tid)
            
    # Detect humans bbox in the frame with detector model.
    detected = detect_model.detect(frame, need_resize=True, expand_bb=10)
    # Predict each tracks bbox of current frame from previous frames information with Kalman filter.
    action_tracker.predict()
    # Merge two source of predicted bbox together.
    for track in action_tracker.tracks:
        det = torch.tensor([track.to_tlbr().tolist() + [0.5, 1.0, 0.0]],
                            dtype=torch.float32)
        detected = torch.cat([detected, det], dim=0) if detected is not None else det
    detections = []  # List of Detections object for tracking.
    if detected is not None:
        # detected = non_max_suppression(detected[None, :], 0.45, 0.2)[0]
        # Predict skeleton pose of each bboxs.
        poses = pose_model.predict(frame, detected[:, 0:4], detected[:, 4])
        # Create Detections object.
        detections = [Detection(kpt2bbox(ps['keypoints'].numpy()),
                                np.concatenate((ps['keypoints'].numpy(),
                                                ps['kp_score'].numpy()), axis=1),
                                ps['kp_score'].mean().numpy()) for ps in poses]
    action_tracker.update(detections)
    # Predict Actions of each track.
    for i, track in enumerate(action_tracker.tracks):
        if not track.is_confirmed():
            continue
        #track_id = track.track_id
        bbox = track.to_tlbr().astype(int)
        #center = track.get_center().astype(int)
        #action = 'pending..'
        #clr = (0, 255, 0)
        # Use 30 frames time-steps to prediction.
        action_name = 'pending...'
        if len(track.keypoints_list) == 30:
            pts = np.array(track.keypoints_list, dtype=np.float32)
            out = action_model.predict(pts, frame.shape[:2])
            action_name = action_model.class_names[out[0].argmax()]
            #action = '{}: {:.2f}%'.format(action_name, out[0].max() * 100)
        if track.time_since_update == 0 and action_name == 'Fall Down':
            print("Falldown is detected! "+str(out[0].max() * 100))
            pid, percentage = is_in_walking_people(walking_people, bbox[0], bbox[1],
                                                    bbox[2], bbox[3], thre=10)
            # fid, f_percentage = is_in_falldown_people(self, falldown_people, bbox[0], bbox[1],
            #                                           bbox[2], bbox[3])
            if pid and not (pid in falldown_people.keys()):
                is_same_pid = 0
                if len(alarms) > 0:
                    for alarm in alarms:
                        if alarm.description == 'falldown' and pid == alarm.track_id:
                            is_same_pid = 1
                if pid and is_same_pid == 0:
                    new_falldown = FalldownAlarm('falldown', frame_id, pid, (
                        bbox[0], bbox[1], bbox[2] - bbox[0], bbox[3] - bbox[1]))
                    new_falldown.cases.append(walking_people[pid])
                    falldown_people[pid] = new_falldown
                    alarms.append(new_falldown)
                    falldown_dt_obj = datetime.datetime.now()
                    falldown_dt_str = falldown_dt_obj.strftime("%y%m%d-%H:%M:%S.%f")
                    print("New Falldown Alarm is detected Time is {0}".format(falldown_dt_str))
                    print("New Falldown Alarm is detected Object id is {0}, frame id is {1}".format(pid, frame_id))
      
    if len(online_ids) > 0:
      original_image_path = os.path.join('../frames', 'original', dt_str + '.jpg')
      tagged_image_path = os.path.join('../frames', 'tagged', dt_str + '.jpg')
      online_im = plot_tracking(frame, online_tlwhs, online_ids, frame_id=frame_id,
                                fps=1. / timer.average_time)
      cv2.imwrite(original_image_path, frame, [int(cv2.IMWRITE_JPEG_QUALITY), 50])
      cv2.imwrite(tagged_image_path, online_im, [int(cv2.IMWRITE_JPEG_QUALITY), 50])
    #전체 영역에 사람이 감지된 경우
    for idx, oid in enumerate(online_ids):
        if in_area(areas, 'general', online_tlwhs[idx]) and not (oid in people.keys()):
            new_loitering = Loitering(oid, dt_obj, frame_id, online_tlwhs[idx], original_image_path, tagged_image_path)
            people[oid] = new_loitering
        elif in_area(areas, 'general', online_tlwhs[idx]) and oid in people.keys():
            if frame_id - people[oid].start_frame >= 30 and not (oid in walking_people.keys()):
                new_walking = Walking(oid, people[oid].start_frame, online_tlwhs[idx], people[oid].original_image_list, people[oid].tagged_image_list)
                walking_people[oid] = new_walking
                print("New Walking Person is detected Object id is {0}, frame is is {1}".format(oid, frame_id))
            if oid in walking_people.keys():
                walking_people[oid].original_image_list.append(original_image_path)
                walking_people[oid].tagged_image_list.append(tagged_image_path)
                walking_people[oid].end_frame = frame_id
                walking_people[oid].tlwhs.append(online_tlwhs[idx])
            if people[oid].end_frame > frame_id - 30:
                people[oid].end_frame = frame_id
                people[oid].tlwhs.append(online_tlwhs[idx])
                people[oid].end_frame = frame_id
                people[oid].tlwhs.append(online_tlwhs[idx])
                people[oid].original_image_list.append(original_image_path)
                people[oid].tagged_image_list.append(tagged_image_path)

        # 배회
        if in_area(areas, 'loitering', online_tlwhs[idx]) and not (
                oid in loitering_people.keys()):  # 처음 배회 감지 영역에 사람이 들어왔을 때
            print("New Loitering is detected Object id is {0}".format(oid))
            new_loitering = Loitering(oid, dt_obj, frame_id, online_tlwhs[idx], original_image_path, tagged_image_path)
            loitering_people[oid] = new_loitering
        elif in_area(areas, 'loitering', online_tlwhs[idx]):  # 이미 감지 영역에 들어온 사람 재인식
            if loitering_people[oid].end_frame >= frame_id - 10:  # 이전 프레임에서 감지한 경우
                loitering_people[oid].end_time = dt_obj
                loitering_people[oid].end_frame = frame_id
                loitering_people[oid].tlwhs.append(online_tlwhs[idx])
                loitering_people[oid].original_image_list.append(original_image_path)
                loitering_people[oid].tagged_image_list.append(tagged_image_path)
                #self.logger.info("Loitering is continued with object is {0}, start frame is {1} and end frame is {2}".format(oid, loitering_people[oid].start_frame, frame_id))
                if loitering_people[oid].end_frame - loitering_people[oid].start_frame >= int(1. / timer.average_time) * 10:  # 10초 이상 배회했을 때 배회 이벤트 발생
                    loitering_dt_obj = datetime.datetime.now()
                    loitering_dt_str = loitering_dt_obj.strftime("%y%m%d-%H:%M:%S.%f")
                    print("New Loitering Alarm is detected Time is {0}".format(loitering_dt_str))
                    print("New Loitering Alarm is detected Object id is {0}".format(oid))
                    loitering_people[oid].is_alarm = True
                    if len(loitering_alarms) == 1:  # 이미 생성된 배회 이벤트가 있을 경우
                        loitering_alarms[0].end_time = loitering_people[oid].end_time
                        loitering_alarms[0].end_frame = loitering_people[oid].end_frame
                        loitering_alarms[0].cases.append(loitering_people[oid])
                        # loitering_alarms[0].tlwhs.append(online_tlwhs[idx])
                    elif len(loitering_alarms) == 0:  # 이미 생성된 배회 이벤트가 없을 경우
                        new_loitering_alarm = LoiteringAlarm('loitering', dt_obj, frame_id)
                        new_loitering_alarm.cases.append(loitering_people[oid])
                        loitering_alarms.append(new_loitering_alarm)
                    else:
                        print("loitering_alarms length is not valid: " + str(len(loitering_alarms)))
        #침입
        if in_area(areas, 'intrusion', online_tlwhs[idx]) and not (oid in intrusion_people.keys()): # 처음 침입 감지 영역에 사람이 들어왔을 때
            new_intrusion = Intrusion(oid, dt_obj, frame_id, online_tlwhs[idx], original_image_path, tagged_image_path)
            new_intrusion.is_alarm = True
            intrusion_people[oid] = new_intrusion
            print("New Intrusion is detected Object id is {0}".format(oid))
            intrusion_dt_obj = datetime.datetime.now()
            intrusion_dt_str = intrusion_dt_obj.strftime("%y%m%d-%H:%M:%S.%f")
            print("New Intrusion is detected Time is {0}".format(intrusion_dt_str))
            if len(intrusion_alarms) == 1: #이미 생성된 침입 이벤트가 있을 경우
                intrusion_alarms[0].cases.append(intrusion_people[oid])
                intrusion_alarms[0].end_time = intrusion_people[oid].end_time
                intrusion_alarms[0].end_frame = intrusion_people[oid].end_frame
                # intrusion_alarms[0].tlwhs.append(online_tlwhs[idx])
            elif len(intrusion_alarms) == 0: #이미 생성된 침입 이벤트가 없을 경우
                new_intrusion_alarm = IntrusionAlarm('intrusion', dt_obj, frame_id)
                new_intrusion_alarm.cases.append(intrusion_people[oid])
                intrusion_alarms.append(new_intrusion_alarm)
            else:
                print("intrusion_alarms length is not valid: " + str(len(intrusion_alarms)))
        elif in_area(areas, 'intrusion', online_tlwhs[idx]): # 이미 감지 영역에 들어온 사람 재인식
            if intrusion_people[oid].end_frame >= frame_id - 30: #이전 프레임에서 감지한 경우
                intrusion_people[oid].end_time = dt_obj
                intrusion_people[oid].end_frame = frame_id
                intrusion_people[oid].tlwhs.append(online_tlwhs[idx])
                intrusion_people[oid].original_image_list.append(original_image_path)
                intrusion_people[oid].tagged_image_list.append(tagged_image_path)
                if len(intrusion_alarms) == 1:
                    intrusion_alarms[0].end_time = dt_obj
                    intrusion_alarms[0].end_frame = frame_id
    #없어진 사람 처리.
    for pid in list(loitering_people.keys()):
        if (not (pid in online_ids) or not in_area(areas, 'loitering',
                                                    online_tlwhs[online_ids.index(pid)])) and \
                loitering_people[pid].end_frame < frame_id - int(1. / timer.average_time) * 1:
            if loitering_people[pid].is_alarm:  # 배회하다가 이상탐지로 이미 탐지된 경우
                if len(loitering_alarms) == 1:
                    is_last = True
                    for case in loitering_alarms[0].cases:  # 제일 마지막으로 없어진 사람인지 확인. 배회와 침입의 경우 가장 마지막 사람을 기준으로 이벤트 발생, 종료됨
                        if case.end_frame == frame_id:
                            is_last = False
                    if is_last:
                        copied_loitering_alarm = copy.deepcopy(loitering_alarms[0])
                        t = threading.Thread(target=file_upload, args=('loitering', copied_loitering_alarm, dt_str))
                        t.start()
                        del loitering_alarms[0]
                else:
                    print("loitering_alarms length is not valid: " + str(len(loitering_alarms)))
                del loitering_people[pid]
            else:  # 10초 미만 배회한 경우
                del loitering_people[pid]
    for pid in list(intrusion_people.keys()):
        if (not(pid in online_ids) or (not in_area(areas, 'intrusion', online_tlwhs[online_ids.index(pid)]))) and intrusion_people[pid].end_frame < frame_id - int(1. / timer.average_time) * 1:
            if len(intrusion_alarms) == 1:
                is_last = True
                for case in intrusion_alarms[0].cases: #제일 마지막으로 없어진 사람인지 확인
                    if case.end_frame == frame_id:
                        is_last = False
                if is_last:
                    copied_intrusion_alarm = copy.deepcopy(intrusion_alarms[0])
                    t = threading.Thread(target=file_upload, args=('intrusion', copied_intrusion_alarm, dt_str))
                    t.start()
                    print("New Intrusion Alarm is detected Object id is {0}".format(pid))
            del intrusion_people[pid]
    for pid in list(people.keys()):
        if not (pid in online_ids) and people[pid].end_frame < frame_id - 50:
            del people[pid]
    for pid in list(walking_people.keys()):
        if not (pid in online_ids) and walking_people[pid].end_frame < frame_id - 50:
            del walking_people[pid]
    for alarm in alarms:
        if alarm.description == 'falldown':
            if len(alarm.image_list) < 50:
                falldown_image_path = os.path.join('../', 'falldown', dt_str + '.jpg')
                cv2.imwrite(falldown_image_path, frame, [int(cv2.IMWRITE_JPEG_QUALITY), 50])
                alarm.image_list.append(falldown_image_path)
            elif len(alarm.image_list) == 50:
                falldown_image_path = os.path.join('../', 'falldown', dt_str + '.jpg')
                cv2.imwrite(falldown_image_path, frame, [int(cv2.IMWRITE_JPEG_QUALITY), 50])
                alarm.image_list.append(falldown_image_path)
                copied_falldown_alarm = copy.deepcopy(alarm)
                t = threading.Thread(target=file_upload, args=('falldown', copied_falldown_alarm, dt_str))
                t.start()
    timer.toc()
    frame_id += 1
  else:
    tries = 0
    max_tries = 100
    while ret == False:
      print("Can't receive frame. Retrying ...")
      cap.release()
      cap = cv2.VideoCapture(rtsp_address)
      ret, frame = cap.read()
      tries += 1
      if tries > max_tries:
        print("Over max tries")
        over_max_tries = 1
        break

2021-03-17 02:21:43 [DEBUG]: ===========Frame 269==========
2021-03-17 02:21:43 [DEBUG]: Activated: []
2021-03-17 02:21:43 [DEBUG]: Refind: []
2021-03-17 02:21:43 [DEBUG]: Lost: []
2021-03-17 02:21:43 [DEBUG]: Removed: []
2021-03-17 02:21:44 [DEBUG]: ===========Frame 270==========
2021-03-17 02:21:44 [DEBUG]: Activated: []
2021-03-17 02:21:44 [DEBUG]: Refind: []
2021-03-17 02:21:44 [DEBUG]: Lost: []
2021-03-17 02:21:44 [DEBUG]: Removed: []


New Intrusion Alarm is detected Object id is 4


2021-03-17 02:21:44 [DEBUG]: ===========Frame 271==========
2021-03-17 02:21:44 [DEBUG]: Activated: []
2021-03-17 02:21:44 [DEBUG]: Refind: []
2021-03-17 02:21:44 [DEBUG]: Lost: []
2021-03-17 02:21:44 [DEBUG]: Removed: []
2021-03-17 02:21:44 [DEBUG]: ===========Frame 272==========
2021-03-17 02:21:44 [DEBUG]: Activated: []
2021-03-17 02:21:44 [DEBUG]: Refind: []
2021-03-17 02:21:44 [DEBUG]: Lost: []
2021-03-17 02:21:44 [DEBUG]: Removed: []
2021-03-17 02:21:44 [DEBUG]: ===========Frame 273==========
2021-03-17 02:21:44 [DEBUG]: Activated: []
2021-03-17 02:21:44 [DEBUG]: Refind: []
2021-03-17 02:21:44 [DEBUG]: Lost: []
2021-03-17 02:21:44 [DEBUG]: Removed: []
2021-03-17 02:21:45 [DEBUG]: ===========Frame 274==========
2021-03-17 02:21:45 [DEBUG]: Activated: []
2021-03-17 02:21:45 [DEBUG]: Refind: []
2021-03-17 02:21:45 [DEBUG]: Lost: []
2021-03-17 02:21:45 [DEBUG]: Removed: []
2021-03-17 02:21:45 [DEBUG]: ===========Frame 275==========
2021-03-17 02:21:45 [DEBUG]: Activated: []
2021-03-1

Falldown is detected! 12.302427738904953
New Falldown Alarm is detected Time is 210317-02:21:48.765973
New Falldown Alarm is detected Object id is 4, frame id is 287


2021-03-17 02:21:49 [DEBUG]: ===========Frame 291==========
2021-03-17 02:21:49 [DEBUG]: Activated: []
2021-03-17 02:21:49 [DEBUG]: Refind: []
2021-03-17 02:21:49 [DEBUG]: Lost: []
2021-03-17 02:21:49 [DEBUG]: Removed: []


...

Processing frame 900 (4.71 fps)


2021-03-17 02:24:31 [DEBUG]: ===========Frame 903==========
2021-03-17 02:24:31 [DEBUG]: Activated: []
2021-03-17 02:24:31 [DEBUG]: Refind: []
2021-03-17 02:24:31 [DEBUG]: Lost: []
2021-03-17 02:24:31 [DEBUG]: Removed: []
2021-03-17 02:24:32 [DEBUG]: ===========Frame 904==========
2021-03-17 02:24:32 [DEBUG]: Activated: []
2021-03-17 02:24:32 [DEBUG]: Refind: []
2021-03-17 02:24:32 [DEBUG]: Lost: []
2021-03-17 02:24:32 [DEBUG]: Removed: []
2021-03-17 02:24:32 [DEBUG]: ===========Frame 905==========
2021-03-17 02:24:32 [DEBUG]: Activated: []
2021-03-17 02:24:32 [DEBUG]: Refind: []
2021-03-17 02:24:32 [DEBUG]: Lost: []
2021-03-17 02:24:32 [DEBUG]: Removed: []
2021-03-17 02:24:32 [DEBUG]: ===========Frame 906==========
2021-03-17 02:24:32 [DEBUG]: Activated: []
2021-03-17 02:24:32 [DEBUG]: Refind: []
2021-03-17 02:24:32 [DEBUG]: Lost: []
2021-03-17 02:24:32 [DEBUG]: Removed: []
2021-03-17 02:24:32 [DEBUG]: ===========Frame 907==========
2021-03-17 02:24:32 [DEBUG]: Activated: []
2021-03-1

Can't receive frame. Retrying ...


2021-03-17 02:24:46 [DEBUG]: ===========Frame 956==========
2021-03-17 02:24:46 [DEBUG]: Activated: []
2021-03-17 02:24:46 [DEBUG]: Refind: []
2021-03-17 02:24:46 [DEBUG]: Lost: []
2021-03-17 02:24:46 [DEBUG]: Removed: []
2021-03-17 02:24:47 [DEBUG]: ===========Frame 957==========
2021-03-17 02:24:47 [DEBUG]: Activated: []
2021-03-17 02:24:47 [DEBUG]: Refind: []
2021-03-17 02:24:47 [DEBUG]: Lost: []
2021-03-17 02:24:47 [DEBUG]: Removed: []
2021-03-17 02:24:47 [DEBUG]: ===========Frame 958==========
2021-03-17 02:24:47 [DEBUG]: Activated: []
2021-03-17 02:24:47 [DEBUG]: Refind: []
2021-03-17 02:24:47 [DEBUG]: Lost: []
2021-03-17 02:24:47 [DEBUG]: Removed: []
2021-03-17 02:24:47 [DEBUG]: ===========Frame 959==========
2021-03-17 02:24:47 [DEBUG]: Activated: []
2021-03-17 02:24:47 [DEBUG]: Refind: []
2021-03-17 02:24:47 [DEBUG]: Lost: []
2021-03-17 02:24:47 [DEBUG]: Removed: []
2021-03-17 02:24:47 [DEBUG]: ===========Frame 960==========
2021-03-17 02:24:47 [DEBUG]: Activated: []
2021-03-1

Processing frame 1000 (4.77 fps)
